### Import Libraries

In [1]:
import duckdb

### Create a in-memory DuckDB Connection

The `allow_unsigned_extensions` setting is particularly important because:<br><br>
By default, DuckDB only loads signed extensions for security
Setting this to `True` allows loading extensions that haven't been cryptographically signed.<br><br>
This can be useful during development or when using custom extensions<br><br>
However, in production environments, you may want to keep this `False` for better security

In [2]:
con = duckdb.connect(config={"allow_unsigned_extensions": True})

#### Install the Geography Extension

In [ ]:
# con.sql("INSTALL 'C:/Users/storl/.duckdb/extensions/v1.4.1/windows_amd64/geography.duckdb_extension'")

#### Load the Geography Extension

In [3]:
con.sql("LOAD geography")

#### Sample GeographyQuery

In [4]:
con.sql("SELECT 'POINT (5.5155307 58.9485486)'::GEOGRAPHY as geo_point")

┌─────────────────────────────────────┐
│              geo_point              │
│              geography              │
├─────────────────────────────────────┤
│ POINT (5.5155307 58.94854860000001) │
└─────────────────────────────────────┘

## Spatial Functions    
<i>https://github.com/paleolimbot/duckdb-geography/blob/main/docs/function-reference.md</i>

##### Unlike the geometry type used with the `Spatial` Extension in DuckDB, the geography type uses a spherical model of the Earth to perform spatial calculations. This means that distance and area calculations are more accurate for large distances and areas, as they take into account the curvature of the Earth.<br>
##### In general, the Geography functions are the same as those implemented in the `Spatial` extension except they are prefixed with `s2_` instead of `st_`.   <br>
##### The Geography Community Extension for DuckDB uses `Google's s2geometry` library via the `s2geography` wrapper library. The S2 integration can be used with R and Python.

##### <i>s2geography wrapper library: https://github.com/paleolimbot/s2geography</i><br>
##### <i>Google's s2geometry library: https://github.com/google/s2geometry</i><br>

-----

### s2_num_points   
This function counts number of points in a (Multi)Polygon

In [5]:
sqltext = """
  SELECT s2_num_points(s2_data_country('Norway')) as norway_num_points,
"""
print(con.sql(sqltext))

┌───────────────────┐
│ norway_num_points │
│       int32       │
├───────────────────┤
│                84 │
└───────────────────┘



### s2_data_city   
This function gets the Point for a city from s2_data_cities()

In [10]:
print(con.sql("SELECT s2_data_city('Oslo') as city"))

┌───────────────────────────────┐
│             city              │
│           geography           │
├───────────────────────────────┤
│ POINT (10.7480333 59.9186361) │
└───────────────────────────────┘



### s2_dwithin    
This function returns `true` if the two geographies are within the specified distance of each other.<br> The distance is in `meters`.

#### Testing the s2_dwithin function - a city within the country's border.

In [11]:
sqltext = """
SELECT s2_dwithin(
  s2_data_city('Oslo'),
  s2_data_country('Norway'),
  30000
) AS is_within;
"""
print(con.sql(sqltext))

┌───────────┐
│ is_within │
│  boolean  │
├───────────┤
│ true      │
└───────────┘



#### Testing distance (30,000 meters) between a city and a different country's border.

In [12]:
sqltext = """
SELECT s2_dwithin(
  s2_data_city('Helsinki'),
  s2_data_country('Estonia'),
  30000
) AS is_within;
"""
print(con.sql(sqltext))

┌───────────┐
│ is_within │
│  boolean  │
├───────────┤
│ false     │
└───────────┘



#### Adjusting the distance to 100,000 meters will change the result of the query.

In [13]:
sqltext = """
SELECT s2_dwithin(
  s2_data_city('Helsinki'),
  s2_data_country('Estonia'),
  100000
) AS is_within;
"""
print(con.sql(sqltext))

┌───────────┐
│ is_within │
│  boolean  │
├───────────┤
│ true      │
└───────────┘



### s2_area
The returned area is in <b>`square meters`</b> as approximated as the area of the polygon on a perfect sphere.

<h4><b>Square meters to square kilometers</b></h4>    Conversion factor: Divide by (1,000,000). 


<h4><b>Square meters to square miles</b></h4>Conversion factor: Divide by (2,589,988).  

#### Find the approximate land area of the country of Denmark in square meters, square kilometers and square miles.<br>
<i>Note:</i> The area is approximated as the area of the polygon on a perfect sphere.</i><br>https://www.google.com/search?q=Denmark+land+area&sca_esv=50bc830c1331a679&rlz=1C1GCEA_enUS1136US1137&biw=1280&bih=630&sxsrf=AE3TifOibZSiF1o9I_MqlGYZzgTYVyIpvg%3A1761492428856&ei=zD3-aKL8M7ru-LYPwdKq6Aw&ved=0ahUKEwiikvzTlsKQAxU6N94AHUGpCs0Q4dUDCBE&uact=5&oq=Denmark+land+area&gs_lp=Egxnd3Mtd2l6LXNlcnAiEURlbm1hcmsgbGFuZCBhcmVhMgoQIxiABBgnGIoFMgYQABgHGB4yBRAAGIAEMgUQABiABDIEEAAYHjIEEAAYHjIEEAAYHjIEEAAYHjIEEAAYHjIEEAAYHkjEHFAAWLsQcAB4AZABAJgBlQGgAcAGqgEDMC43uAEDyAEA-AEBmAIHoAKZB8ICBxAjGLACGCfCAggQABgHGAoYHpgDAJIHAzAuN6AHnzWyBwMwLje4B5kHwgcDMy03yAdM&sclient=gws-wiz-serp

In [14]:
sqltext = """
SELECT printf('%.0f',s2_area(s2_data_country('Denmark'))) AS "area(m2)",
       printf('%.0f',s2_area(s2_data_country('Denmark')) / 1e6) AS "area(km2)",
       printf('%.0f',s2_area(s2_data_country('Denmark')) / 2.589988e6) AS "area(mi2)"
"""
print(con.sql(sqltext))

┌─────────────┬───────────┬───────────┐
│  area(m2)   │ area(km2) │ area(mi2) │
│   varchar   │  varchar  │  varchar  │
├─────────────┼───────────┼───────────┤
│ 42557165050 │ 42557     │ 16431     │
└─────────────┴───────────┴───────────┘



### s2_contains    
Returns `true` if the first geography contains the second.

In [15]:
sqltext = """
SELECT s2_contains(s2_data_country('Estonia'), s2_data_city('Oslo')) As Oslo_in_Estonia;
"""
print(con.sql(sqltext))

┌─────────────────┐
│ Oslo_in_Estonia │
│     boolean     │
├─────────────────┤
│ false           │
└─────────────────┘



In [16]:
sqltext = """
SELECT s2_contains(s2_data_country('Estonia'), s2_data_city('Tallinn')) As Tallinn_in_Estonia;
"""
print(con.sql(sqltext))

┌────────────────────┐
│ Tallinn_in_Estonia │
│      boolean       │
├────────────────────┤
│ true               │
└────────────────────┘



### s2_x,  s2_y   
These functions returns the X and Y coordinates respectively of a Point geography.

In [17]:
sqltextXY = """
SELECT s2_x(s2_data_city('Oslo')::GEOGRAPHY) as x_coord,
       s2_y(s2_data_city('Oslo')::GEOGRAPHY) as y_coord
"""
print(con.sql(sqltextXY))

┌────────────┬────────────┐
│  x_coord   │  y_coord   │
│   double   │   double   │
├────────────┼────────────┤
│ 10.7480333 │ 59.9186361 │
└────────────┴────────────┘



### s2_box_intersects   
This function returns `true` if the bounding boxes of the two geographies intersect.

In [18]:
sqltext = """
SELECT s2_box_intersects(
  s2_bounds_box(s2_data_country('Norway')),
  s2_bounds_box(s2_data_country('Sweden'))
) as CTY_Intersect
"""
con.sql(sqltext)

┌───────────────┐
│ CTY_Intersect │
│    boolean    │
├───────────────┤
│ true          │
└───────────────┘

In [19]:
sqltext = """
SELECT s2_box_intersects(
  s2_bounds_box(s2_data_country('Italy')),
  s2_bounds_box(s2_data_country('Norway'))
) as CTY_Intersect
"""
print(con.sql(sqltext))

┌───────────────┐
│ CTY_Intersect │
│    boolean    │
├───────────────┤
│ false         │
└───────────────┘



#### Displaying Geography data in VSCode<br>
To visualize the Geography data in VSCode, you can use the `Geo Data Viewer` extension.<br><br>
This extension allows you to visualize geographic data directly within VSCode, making it easier to analyze and interpret spatial information.<br><br>
You can install the `Geo Data Viewer` extension from the VSCode marketplace and use it to view the results of your geography queries in a more intuitive way.<br><br>
Geo Data Analytics tool for VSCode IDE with kepler.gl support to generate and view maps from geospatial data files.

#### Load Census_Illinois_State spatial data from Github and save as a JSON file

In [20]:
sqltextC = """
select * from read_json('https://raw.githubusercontent.com/Tor-Storli/Geospatial_Data/refs/heads/main/data/IL_Counties_Census.json')
"""
con.sql(f"COPY({sqltextC}) TO 'IL.json' (FORMAT JSON)")


#### Create a CSV file from s2_data_countries() and s2_data_cities() 

In [21]:
sqlCountryCity = """
    SELECT countries.name as country, cities.name as city, cities.geog as geogCity,
    countries.geog as geogCountry
    FROM s2_data_countries() AS countries
    INNER JOIN s2_data_cities() AS cities
    ON s2_intersects(countries.geog, cities.geog)
    WHERE countries.name IN ('Norway', 'Sweden', 'Finland')
"""
countrycities_csv = con.sql(f"COPY({sqlCountryCity}) TO 'country-cities.csv' (FORMAT CSV)")

#### View data from created CSV file using DuckDB's `read_csv` function<br> and create a pandas dataframe from it

In [22]:
df_countries = con.sql("SELECT * FROM read_csv('country-cities.csv')").df()
df_countries.head()

,country,city,geogCity,geogCountry
0,Finland,Helsinki,POINT (24.932180499999998 60.1775092),"POLYGON ((28.59193 69.064777, 29.0155730000000..."
1,Norway,Oslo,POINT (10.7480333 59.9186361),"MULTIPOLYGON (((28.165547 71.185474, 26.37005 ..."
2,Sweden,Stockholm,POINT (18.0953889 59.3527058),"POLYGON ((22.183174000000005 65.723741, 23.903..."


<center>==================== END OF SCRIPT ====================</center >